<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/exercicio_svm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split


In [24]:
# Data understanding
DATA = pd.read_csv('/content/regressao_Q2.csv')
DATA.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,target
0,1.92864,1.48414,0.86814,-0.67666,-0.28747,-1.45108,-0.73662,0.03134,-0.53872,1.30562,0.11557,-0.30478,127.682465
1,0.22185,-0.55320,-0.29845,0.65870,-0.30132,1.49319,-0.43096,0.33835,-0.30827,1.25765,1.88584,-0.57726,50.022972
2,-0.02183,0.13602,-0.37426,-1.29096,0.71912,1.95088,1.99309,-1.24197,-2.15377,-2.01455,-0.84625,0.29845,-24.364369
3,0.86528,1.36937,1.27999,1.18124,-0.72465,-0.02175,0.40340,-0.28272,-0.44390,0.84051,0.03326,-0.98550,96.415408
4,1.41333,-0.02920,-0.67228,1.76116,-1.12178,0.18002,0.48476,-0.76394,-0.16421,-2.42048,0.79727,-0.44033,157.407129


In [25]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,v_1,float64,0,0.0,2491
1,v_2,float64,0,0.0,2489
2,v_3,float64,0,0.0,2489
3,v_4,float64,0,0.0,2488
4,v_5,float64,0,0.0,2484
5,v_6,float64,0,0.0,2489
6,v_7,float64,0,0.0,2489
7,v_8,float64,0,0.0,2492
8,v_9,float64,0,0.0,2495
9,v_10,float64,0,0.0,2491


In [26]:
DATA.describe()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,target
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,0.010030,-0.060168,0.009921,-0.010504,0.000397,-0.006430,0.015629,0.000075,-0.001657,0.000290,-0.023323,0.018136,0.555105
std,0.998695,0.987547,1.003915,1.015696,1.006447,0.979081,1.000192,1.005445,1.009637,1.005458,1.008678,1.008743,143.720826
min,-3.974510,-3.613150,-3.558150,-3.613370,-3.065630,-2.968330,-3.355500,-3.682700,-3.770810,-3.978460,-3.614730,-4.087600,-471.528079
25%,-0.624668,-0.713672,-0.667495,-0.697965,-0.694120,-0.678975,-0.651733,-0.669007,-0.678168,-0.691798,-0.698605,-0.630763,-96.214263
50%,-0.011855,-0.048880,0.030565,-0.008475,-0.003535,-0.016775,0.028450,-0.005685,0.001500,0.017255,0.003330,0.022010,0.217178
75%,0.642370,0.595177,0.689033,0.670802,0.669773,0.648637,0.695677,0.719817,0.672083,0.675290,0.669048,0.703172,97.602166
max,3.433680,3.550180,3.167250,3.592510,3.161000,3.324870,4.082660,3.143080,3.669470,3.143360,3.566110,3.233960,556.202966


In [27]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR

x_treino_final = DATA.drop('target', axis=1)
y_treino_final = DATA['target']


# Definindo o modelo
model = SVR()

# Definindo os parâmetros para o grid search
param_grid = {
    'C': [0.001],
    'kernel': ['linear'],
    # 'degree': [2, 3],  # Só é relevante se kernel='poly'
    # 'gamma': [0.1, 1]
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=KFold(n_splits = 5, shuffle=False), # padrão de shuffle é False
                    verbose=2,
                    scoring='neg_mean_squared_error',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1,
                    return_train_score = True)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor mse: ", grid.best_score_)




best_model = grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhores Parâmetros:  {'C': 0.001, 'kernel': 'linear'}
Melhor mse:  -20207.880173137346


In [28]:
estudo = pd.DataFrame(grid.cv_results_ )
estudo['mean_train_score']

0   -20199.036264
Name: mean_train_score, dtype: float64

In [29]:
estudo['mean_test_score']

0   -20207.880173
Name: mean_test_score, dtype: float64